In [1]:
import requests 
from collections import defaultdict

In [2]:
docs_url = 'https://raw.githubusercontent.com/AdairPonceuwu/ch_llm/main/documents.json'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
documents[0]

{'text': 'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.',
 'section': 'Vinculación a otros instrumentos municipales',
 'question': '¿Cómo contribuirá el Eje 4 a la protección del patrimonio histórico cultural del Centro Histórico?',
 'topic': 'PMD'}

In [4]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['topic']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [6]:
documents[3]

{'text': 'En el último decenio, la zona metropolitana registró su menor crecimiento poblacional con una tasa media anual de 0.9%. Esta situación contrasta con las altas tasas de crecimiento de municipios como Cuautlancingo, San Andrés Cholula, Huejotzingo y Coronango.',
 'section': 'Densidad poblacional',
 'question': '¿Cuál fue la tasa media anual de crecimiento poblacional en la zona metropolitana en el último decenio?',
 'topic': 'PMD',
 'id': 'bbcc69fc'}

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
len(hashes), len(documents)

(186, 186)

In [9]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [10]:
hashes['c78f3aa6']

[{'text': 'C6 Reservas ecológica, parques, jardines, espacio públcio y área verde',
  'section': 'Intervención Arquitectónica ',
  'question': '¿Qué establece la categoría 6 en el Programa Parcial de Desarrollo Urbano Sustentable?',
  'topic': 'PPDUS',
  'id': 'c78f3aa6'}]

In [11]:
import json

In [12]:
with open('documents-with-ids.json', 'w', encoding='utf-8') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=2)

In [74]:
prompt_template = """
Eres una persona que habla español y necesita informacion sobre el Centro Histórico de Puebla.
Formula 5 preguntas que esta persona puede preguntar con base a un registro FAQ. Las preguntas deben de estar formuladas
con base a la respuesta que tiene cada pregunta de cada registro y las preguntas deben de ser completas y no tan cortas.
Si es posible, usa la menor cantidad posible de palabras del registro. 

El registro:

section: {section}
question: {question}
answer: {text}



Tu respuesta debe de estar de la siguiente forma para que sea "parsable JSON" sin utilizar "code blocks", sigue el sgiueinte formato:

[\n "pregunta1", \n "pregunta2", \n "pregunta3", \n "pregunta4", \n "pregunta5"\n]

No necesito que agreges texto introductorio a la repuesta, siempre cierra el corchete y pon un espacio despues de cada "\n"
""".strip()

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='llama3.1',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [83]:
doc = documents[8]
questions = generate_questions(doc)

In [84]:
questions

'[\n "¿Qué factores han contribuido al despoblamiento del Centro Histórico?",\n "¿Cuál es la principal causa de insuficientes recursos invertidos en el patrimonio edificado del Centro Histórico?",\n "¿Cómo se puede clasificar el nivel de deterioro del patrimonio edificado en el Centro Histórico según la información proporcionada?",\n "¿Se han realizado acciones para revertir los efectos del despoblamiento y bajo inversión en el patrimonio del Centro Histórico?",\n "¿Hay algún esfuerzo coordinado entre la comunidad privada y gubernamental para preservar los bienes culturales y físicos dentro del Centro Histórico?"\n]'

In [85]:
json.loads(questions)

['¿Qué factores han contribuido al despoblamiento del Centro Histórico?',
 '¿Cuál es la principal causa de insuficientes recursos invertidos en el patrimonio edificado del Centro Histórico?',
 '¿Cómo se puede clasificar el nivel de deterioro del patrimonio edificado en el Centro Histórico según la información proporcionada?',
 '¿Se han realizado acciones para revertir los efectos del despoblamiento y bajo inversión en el patrimonio del Centro Histórico?',
 '¿Hay algún esfuerzo coordinado entre la comunidad privada y gubernamental para preservar los bienes culturales y físicos dentro del Centro Histórico?']

In [19]:
from tqdm.auto import tqdm

D:\Anaconda\envs\centro_historico\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
results = {}

In [21]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████████████████████████████████████████████████████| 186/186 [2:07:42<00:00, 41.20s/it]


In [59]:
results[doc_id]

'[\n"¿Qué reconocimiento recibió el Archivo General Municipal de Puebla y cómo se relaciona con el Centro Histórico de Puebla?,"\n"¿Cuáles son las series y colecciones del Archivo General Municipal de Puebla que fueron inscritas en 2015?",\n"¿Pertenece al Centro Histórico de Puebla algún edificio o estructura con importancia documental como el Archivo General Municipal?",\n"¿Se pueden realizar visitas guiadas por las instalaciones del Archivo General Municipal como parte de un recorrido turístico en el Centro Histórico?",\n"¿Qué actividades culturales y eventos destacados ocurren en el Centro Histórico de Puebla, especialmente en relación con el reconocimiento recibido por el Archivo General Municipal?"\n]'

In [49]:
results

{'335b2f94': '[\n  "¿Cuáles son los planes para repoblar y mejorar urbanísticamente la zona del Centro Histórico tras la implementación del Eje 4?",\n  "¿Qué medidas específicas habrá para proteger el patrimonio histórico cultural en las áreas de mejoramiento urbano dentro del Centro Histórico de Puebla?",\n  "¿Cómo se garantiza que los esfuerzos de repoblamiento y mejora urbana no dañen o comprometan la esencia patrimonial del lugar?",\n  "¿Se incluirán iniciativas dirigidas a vislumbrar formas innovadoras para el uso y disfrute del espaciopúblico histórico, combinando restauración preservativa y reactividades económicas?",\n  "¿Es posible que exista planificación en relación con la integración físico-material urbana, para apoyar la revitalización de este histórico entorno cultural?"\n]',
 '805017f9': '[\n "¿Qué objetivos específicos tiene el Plan de Manejo del Centro Histórico de Puebla y qué tan comprometidas están los lugareños en su gestión y conservación?",\n "¿Cuáles son algunas

In [52]:
json_string = '''[
  "¿Cuál es la importancia del Centro Histórico de Puebla en la ciudad y cómo afecta su economía en cuanto al turismo?",
  "¿Son admisibles personas con alguna discapacidad dentro de las áreas protegidas por el Centro Histórico de Puebla, como el Zócalo o la Catedral?",
  "¿Cuáles son los horarios más tranquilos para visitar lugares emblemáticos en el centro histórico de Puebla?",
  "¿Por qué es necesario tomar precauciones al explorar los edificios y calles del Centro Histórico de Puebla, además de seguir las indicaciones turísticas?",
  "¿Cómo afecta la UNESCO su patrimonio cultural sobre este lugar en comparación a otras ciudades?"
]'''

In [57]:
results["3b870dae"] = json.dumps(json_string, ensure_ascii=False)

In [58]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    print("a")
    parsed_resulst[doc_id] = json.loads(json_questions)

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


JSONDecodeError: Expecting ',' delimiter: line 3 column 1 (char 127)

In [27]:
import pickle

with open('results.pkl', 'wb') as file:
    pickle.dump(results, file)


In [28]:
# Cargar la variable desde el archivo binario
with open('results.pkl', 'rb') as file:
    loaded_results = pickle.load(file)




{'335b2f94': '[\n  "¿Cuáles son los planes para repoblar y mejorar urbanísticamente la zona del Centro Histórico tras la implementación del Eje 4?",\n  "¿Qué medidas específicas habrá para proteger el patrimonio histórico cultural en las áreas de mejoramiento urbano dentro del Centro Histórico de Puebla?",\n  "¿Cómo se garantiza que los esfuerzos de repoblamiento y mejora urbana no dañen o comprometan la esencia patrimonial del lugar?",\n  "¿Se incluirán iniciativas dirigidas a vislumbrar formas innovadoras para el uso y disfrute del espaciopúblico histórico, combinando restauración preservativa y reactividades económicas?",\n  "¿Es posible que exista planificación en relación con la integración físico-material urbana, para apoyar la revitalización de este histórico entorno cultural?"\n]', '805017f9': '[\n "¿Qué objetivos específicos tiene el Plan de Manejo del Centro Histórico de Puebla y qué tan comprometidas están los lugareños en su gestión y conservación?",\n "¿Cuáles son algunas 

In [30]:
loaded_results

{'335b2f94': '[\n  "¿Cuáles son los planes para repoblar y mejorar urbanísticamente la zona del Centro Histórico tras la implementación del Eje 4?",\n  "¿Qué medidas específicas habrá para proteger el patrimonio histórico cultural en las áreas de mejoramiento urbano dentro del Centro Histórico de Puebla?",\n  "¿Cómo se garantiza que los esfuerzos de repoblamiento y mejora urbana no dañen o comprometan la esencia patrimonial del lugar?",\n  "¿Se incluirán iniciativas dirigidas a vislumbrar formas innovadoras para el uso y disfrute del espaciopúblico histórico, combinando restauración preservativa y reactividades económicas?",\n  "¿Es posible que exista planificación en relación con la integración físico-material urbana, para apoyar la revitalización de este histórico entorno cultural?"\n]',
 '805017f9': '[\n "¿Qué objetivos específicos tiene el Plan de Manejo del Centro Histórico de Puebla y qué tan comprometidas están los lugareños en su gestión y conservación?",\n "¿Cuáles son algunas

In [29]:
parsed_resulst = {}

for doc_id, json_questions in loaded_results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

JSONDecodeError: Invalid control character at: line 3 column 148 (char 268)